In [1]:
%pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Self attention

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split



In [6]:
# Disable GPU usage if GPU is not available or not needed
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [7]:

# Load and resize images from a directory with folder names as labels
def load_and_resize_images_from_folder(folder, target_size=(300, 300)):
    images = []
    labels = []
    class_names = []

    # Check if the folder path is valid
    if not os.path.isdir(folder):
        raise ValueError(f"Invalid folder path: {folder}")

    # Get the list of class names (subfolder names)
    class_names = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

    # Check if any class subfolders exist
    if not class_names:
        raise ValueError(f"No class subfolders found in {folder}")

    for i, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize image to target size
                img = cv2.resize(img, target_size)
                images.append(img)
                labels.append(i)  # Assign the class index as the label

    # Check if any images were loaded
    if not images:
        raise ValueError(f"No images found in {folder}")

    return images, labels, class_names



In [8]:
#mount google drive to read data
folder = 'RiceLeafDiseaseImages'

import pandas as pd
from google.colab import data_table
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/DSBA6165_Group_Project/'+ folder



Mounted at /content/drive


In [9]:
# Define path to the folder containing images
#folder_path = "/DSBA6165/Group Project/RiceLeafDiseaseImages"
#folder_path = "/content/drive/MyDrive/DSBA6165_Group_Project/RiceLeafDiseaseImages"



try:
    # Load and resize images
    images, labels, class_names = load_and_resize_images_from_folder(path)
except ValueError as e:
    print(f"Error: {e}")
    exit()



In [10]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Define the number of classes
num_classes = len(class_names)

# Define input shape
input_shape = (300, 300, 3)  # Assuming resized images are 300x300 pixels with 3 channels (RGB)

# Rest of the code remains the same...

In [11]:
# Define your CNN-based self-attestation mechanism model
def create_self_attestation_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [12]:

# Create the model
self_attestation_model = create_self_attestation_model(input_shape, num_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [13]:
# Compile the model
self_attestation_model.compile(optimizer='adam',
                               loss='sparse_categorical_crossentropy',
                               metrics=['accuracy'])

In [ ]:
# Train the model
self_attestation_model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
test_loss, test_accuracy = self_attestation_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')